# Section 2-2 - Regularized Neural Network

In this section, we'll introduce two additional layers to our model. The first is called the rectified linear unit (hereafter, ReLU), which helps introduce non-linearity into the network. The second is called the dropout layer, which acts to regularize the network and prevent overfitting.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/mnist.csv')

In [2]:
df_train = df.iloc[:33600, :]

X_train = df_train.iloc[:, 1:].values / 255.
y_train = df_train['label'].values
y_train_onehot = pd.get_dummies(df_train['label']).values

In [3]:
df_test = df.iloc[33600:, :]

X_test = df_test.iloc[:, 1:].values / 255.
y_test = df_test['label'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['label'].values)

y_prediction = model.predict(X_test)
print("\naccuracy", np.sum(y_prediction == df_test['label'].values) / float(len(y_test)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   19.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



accuracy 0.965


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


## Regularized NN

While matrix operations are linear, there could be a non-linear relationship between the features and the label. Introducing a ReLU layer, defined as f(x) = max(0, x), can help the model capture this interaction. ReLU is widely used as its simplicity allows for much faster training without a high cost to accuracy.

The dropout layer can be thought of as a form of sampling, where output values are randomly set to zero by a pre-specified probability. This creates a more robust network as the process prevents interdependence, and as such the model is less likely to overfit on the training data. It is surprisingly effective, which has made it an active area of research.

Following Andrej Karpathy's advice of "don't be a hero", the example shown here is from the Keras repository.

In [5]:
# https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop

start = time()

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=12)

model.summary()

print('\ntime taken %s seconds' % str(time() - start))

2023-03-24 10:37:54.651495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/12
1050/1050 [==============================] - 5s 4ms/step - loss: 0.2653 - accuracy: 0.9188
Epoch 2/12
1050/1050 [==============================] - 5s 5ms/step - loss: 0.1267 - accuracy: 0.9634
Epoch 3/12
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0960 - accuracy: 0.9729
Epoch 4/12
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0808 - accuracy: 0.9770
Epoch 5/12
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0695 - accuracy: 0.9810
Epoch 6/12
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0618 - accuracy: 0.9822
Epoch 7/12
1050/1050 [==============================] - 6s 5ms/step - loss: 0.0523 - accuracy: 0.9857
Epoch 8/12
1050/1050 [==============================] - 6s 6ms/step - loss: 0.0462 - accuracy: 0.9868
Epoch 9/12
1050/1050 [==============================] - 6s 6ms/step - loss: 0.0409 - accuracy: 0.9893
Epoch 10/12
1050/1050 [==============================] - 6s 6ms/step - loss: 0.037

In [6]:
y_prediction = np.argmax(model.predict(X_test), axis=-1)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

263/263 [==============================] - 0s 1ms/step

accuracy 0.9783333333333334


Introducing ReLU and dropout layers have enabled the model to outperform the benchmark by 1%! In the next section, we introduce new layers that take advantage of the 2D structure of the image to further improve model performance.